In [1]:
import math
from itertools import combinations

In [2]:
import pandas as pd

In [3]:
import gensim.downloader as api
import json
import pickle
import time
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

In [4]:
df = pd.read_csv("abstracts_stunting_in_children.csv", encoding='unicode_escape')

In [5]:
df.head()
abstract = list(df['Abstract'])

In [6]:
word_search = pd.read_csv("Word_search_nfhs_cnns.csv")

In [7]:
search_word_nfhs = list(word_search['search_word_nfhs'])
# search_word_nfhs

In [8]:
def npmi_memoing(wi, wj, wiki_corpus, keywords_counts):
    def count_word(w, wiki_corpus):
        w_ct = 0
        w_idx = []
        for i, doc in enumerate(wiki_corpus):
            if w in doc:
                w_ct += 1
                w_idx.append(i)
        keywords_counts[w] = (w_ct, w_idx)
        
    # count word occurance if not in keywords_counts
    if wi not in keywords_counts and wj not in keywords_counts:
        wi_ct = 0
        wi_idx = []
        wj_ct = 0
        wj_idx = []
        for i, doc in enumerate(wiki_corpus):
            if wi in doc:
                wi_ct += 1
                wi_idx.append(i)
            if wj in doc:
                wj_ct += 1
                wj_idx.append(i)
        keywords_counts[wi] = (wi_ct, wi_idx)
        keywords_counts[wj] = (wj_ct, wj_idx)
    elif wi not in keywords_counts:
        count_word(wi, wiki_corpus)
    elif wj not in keywords_counts:
        count_word(wj, wiki_corpus)
            
    # count both
    p_wi_wj = len(set(set(keywords_counts[wi][1]) & set(keywords_counts[wj][1])))
        
    if p_wi_wj == 0: # they never occur together
        return -1.0
    else:
        p_wi = keywords_counts[wi][0]
        p_wj = keywords_counts[wj][0]
        
        p_wi /= FREQ
        p_wj /= FREQ
        p_wi_wj /= FREQ

        numer = math.log(p_wi_wj / (p_wi * p_wj))
        denom = -math.log(p_wi_wj)
        return numer / denom

In [9]:
FREQ = sum([len(doc) for doc in abstract])
print(FREQ)

511273


In [10]:
combs = combinations(search_word_nfhs, 2)
keywords_counts = {}
for c in combs:
    res = npmi_memoing(c[0],c[1],abstract, keywords_counts)
    print(c[0],c[1], res)

educated child birth -1.0
educated household electricity -1.0
educated improved drinking water -1.0
educated improved sanitation facility -1.0
educated clean cooking fuel -1.0
educated iodized salt -1.0
educated health insurance -1.0
educated child attended school -1.0
educated illiterate 0.7939809464327944
educated birth order -1.0
educated mother pregnant -1.0
educated hygienic methods during  menstural period -1.0
educated family planning -1.0
educated female sterilization -1.0
educated male sterilization -1.0
educated IUD or PPIUD  -1.0
educated pill -1.0
educated condom -1.0
educated injections -1.0
educated no family planning -1.0
educated mother antenatal checkup -1.0
educated mother antenatal care -1.0
educated neonatal tetanus -1.0
educated iron 0.5784399156724092
educated folic acid -1.0
educated institutional birth -1.0
educated oral rehydration -1.0
educated zinc -1.0
educated diarrhoea 0.7023868771770148
educated respiratory infection -1.0
educated fever -1.0
educated brea

In [ ]:
keywords_counts

In [ ]:
keywords_columns = {}
keywords_sums = {}
def calculate_npmi_wiki_keywords(keywords):
    ret = []
    keywords_updated = []
    for keyword in keywords:
        if keyword not in keywords_columns:
            try:
                w_id = wiki_features.index(keyword)
            except:
                continue
            keywords_updated.append(keyword)
            w_arr = wiki_matrix[:,w_id].A.flatten()
            p_w = w_arr.sum()
            keywords_columns[keyword] = w_arr
            keywords_sums[keyword] = p_w
        else:
            keywords_updated.append(keyword)
    # generate combinations
    comb = list(combinations(keywords_updated, 2))
        
    for i, (w1, w2) in enumerate(comb):
        w1_arr = keywords_columns[w1]
        w2_arr = keywords_columns[w2]
        p_w1_w2 = ((w1_arr == 1) & (w1_arr == w2_arr)).sum() 
        
        if p_w1_w2 == 0: # they never occur together
            res = -1.0
        else:
            p_w1 = keywords_sums[w1]
            p_w2 = keywords_sums[w2]

            p_w1 /= wiki_corpus_size
            p_w2 /= wiki_corpus_size
            p_w1_w2 /= wiki_corpus_size
            num = math.log(p_w1_w2 / (p_w1 * p_w2))
            res = num / -math.log(p_w1_w2)
        ret.append((w1, w2, res))
    return ret

In [ ]:
def vanilla_npmi(wi, wj, wiki_corpus):
    p_wi = p_wj = p_wi_wj = 0
    for doc in wiki_corpus:
        if wi in doc:
            p_wi += 1
        if wj in doc:
            p_wj += 1
        if wi in doc and wj in doc:
            p_wi_wj += 1
            
    p_wi /= FREQ
    p_wj /= FREQ
    p_wi_wj /= FREQ
    
    numer = math.log(p_wi_wj / (p_wi * p_wj))
    denom = -math.log(p_wi_wj, 2)
    return numer / denom

In [ ]:
combs = combinations(search_word_nfhs, 2)
for c in combs:
    res = vanilla_npmi(c[0],c[1],abstract)
    print(c[0],c[1], res)

In [ ]:
matrix = []
matrix = calculate_npmi_wiki_keywords(search_word_nfhs)

In [ ]:
matrix